![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

# Master THesis: Tucapital Scrapping

## Scraping with Selenium + Beautiful Soup; Regex with html_element text preprocess

Selemnium is able to simulate human-like action in webpage.<br>
Beautiful Soup is able to fetch entire or selected html page, we directly operate search and select text on that.<br>
We use Regex to operate text preprocess, our case is regex on html_element<br>

In [42]:
#import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
#import requests

import os
import pandas as pd
from datetime import datetime
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from tqdm.auto import tqdm #progress bar
# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

c:\Users\wuhangze\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    #profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,firefox_binary_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    options.binary_location = firefox_binary_path #binary location
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [24]:
# start selenium
dfolder='D:/Hangze Wu/BSE/DSDM/semester3/Master Thesis/thesis project/downloads/'
geko_path='D:/Hangze Wu\BSE/DSDM/semester3/Master Thesis/geckodriver.exe'
# Tucapital
link='https://www.tucapital.es/blogs/noticias/'
save_file_path ="/tucapital-news_links.csv"

browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path,firefox_binary_path = "C:/Program Files/Mozilla Firefox/firefox.exe")
browser.get(link)

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:3: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:3: SyntaxWarning: invalid escape sequence '\B'
C:\Users\wuhangze\AppData\Local\Temp\ipykernel_15476\768619607.py:2: SyntaxWarning: invalid escape sequence '\B'
  dfolder='D:/Hangze Wu\BSE/DSDM/semester3/Master Thesis/thesis project/downloads/'
C:\Users\wuhangze\AppData\Local\Temp\ipykernel_15476\768619607.py:3: SyntaxWarning: invalid escape sequence '\B'
  geko_path='D:/Hangze Wu\BSE/DSDM/semester3/Master Thesis/geckodriver.exe'


# Click on Refuse Cookies
Refuse all cookies

In [15]:
refuse_cookies='//*[@id="onetrust-reject-all-handler"]'
check_and_click(browser, xpath = refuse_cookies, type = 'xpath')

In [15]:
browser.refresh()

# Click on Cookies Option Page
Click to open cookies option page

In [17]:
review_page='button.f88a5204c2:nth-child(3)'
check_and_click(browser, xpath = review_page, type = 'css')

# Scrapping News

1. Nevigate to News Setion in Selenium
2. Use Beautiful Soup to fetch and save in reviews variavle
3. loop to extrac review, date, name, rating, travel category, country, helpful_tag, translate_require
4. click to select next page (we ++index to loop over pages and pick page button by index)
5. -> repeat till end

In [35]:
# Function to compare dates
def is_date_greater(current_date, cutoff_date):
    from datetime import datetime
    date_format = "%d/%m/%Y"
    return datetime.strptime(current_date, date_format) > datetime.strptime(cutoff_date, date_format)
# Function to check whether is valid date
def is_date_within_cutoff(first_item, cutoff_date, date_pattern):
    # check wehter inside cutoff date is valid
    title_container = first_item.find('div', class_='noticias_list_item_title')
    is_valid_date = False
    if title_container:
        text = title_container.get_text()
        date_match = date_pattern.search(text)
        if date_match and is_date_greater(date_match.group(1), cutoff_date):
            # Process the full page because the first item's date is greater
            print(f"Processing page - First item date: {date_match.group(1)}")
            is_valid_date = True
        else:
            print("Cutoff date reached, stopping navigation.")
            is_valid_date = False
        return is_valid_date
# Function to get current page's news items
def get_links_date(items, date_pattern):
    news_items = []
    for item in items:
        title_container = item.find('div', class_='noticias_list_item_title')
        if title_container:
            link = title_container.find('a')
            title = link.text.strip() if link else None
            url = link.get('href', None)

            # Extract the date using regex
            # Assuming the date is in the text of `title_container`
            text = title_container.get_text()
            date_match = date_pattern.search(text)
            date = date_match.group(0) if date_match else 'No date'

            news_items.append({
                'title': title,
                'url': url,
                'date': date
            })
    return news_items

In [37]:
cutoff_date = '01/01/2021'
# Re pattern to '-DD/MM/YYYY:'
date_pattern = re.compile(r'(\d{2}/\d{2}/\d{4})')
news_items = []

# iterate page to extract news_link items
try:
    while True:
        # Mount to items
        html_content = browser.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        # Find all 'div' elements with the class 'noticias_list_item'
        items = soup.find_all('div', class_='noticias_list_item')

        first_item = items[0]

        is_valid_date = is_date_within_cutoff(first_item = first_item, cutoff_date = cutoff_date, date_pattern = date_pattern)
        if is_valid_date:
            current_page_items = get_links_date(items, date_pattern)
            news_items.extend(current_page_items)
            # Print extracted information
            for news in current_page_items:
                print(f"Date: {news['date']}, Title: {news['title']}, URL: {news['url']}")
        else:
            break
        
        #wait to refresh
        time.sleep(1.3)
        # Navigate to next page
        next_page_button='a.nextpostslink'
        check_and_click(browser, xpath = next_page_button, type = 'css')

# save to local csv file
finally:
    print("Finally")
    news_df = pd.DataFrame(news_items)
    #save
    news_df.to_csv(dfolder + save_file_path, index=False)

    #browser.quit()

Processing page - First item date: 23/04/2024
Date: 23/04/2024, Title: Mañana, último día de la cuenta al 2,89% TAE y depósito al 3,44% TAE de Renault Bank, URL: https://www.tucapital.es/depositos/manana-ultimo-dia-de-la-cuenta-al-289-tae-y-deposito-al-344-tae-de-renault-bank/
Date: 22/04/2024, Title: ING te da hasta 120€ si domicilias nómina y pagas con su tarjeta, URL: https://www.tucapital.es/cuentas/ing-te-da-hasta-120e-si-domicilias-nomina-y-pagas-con-su-tarjeta/
Date: 22/04/2024, Title: Depósitos de Banco BIG: ligera subida en los normales, y combinados 5,06% TAE, URL: https://www.tucapital.es/depositos/depositos-de-banco-big-ligera-subida-los-normales-y-combinados-506-tae/
Date: 22/04/2024, Title: El futuro de Bitcoin: predicciones para los próximos 5 años, URL: https://www.tucapital.es/blogs/guias/el-futuro-de-bitcoin-predicciones-para-los-proximos-5-anos/
Date: 19/04/2024, Title: Eco: Cetelem sólo permite transferencias máximas diarias de 30.000€, URL: https://www.tucapital.es

# Scrapping Text by link
1.Iterate over the DataFrame rows.</br>
2.For each row, check if the URL and title are not None.</br>
3.Use Selenium to navigate to the URL.</br>
4.Use BeautifulSoup to parse the page and extract the desired text.</br>

In [45]:
df = pd.read_csv(dfolder + save_file_path)
df.head()

,title,url,date
0,"Mañana, último día de la cuenta al 2,89% TAE y...",https://www.tucapital.es/depositos/manana-ulti...,23/04/2024
1,ING te da hasta 120€ si domicilias nómina y pa...,https://www.tucapital.es/cuentas/ing-te-da-has...,22/04/2024
2,Depósitos de Banco BIG: ligera subida en los n...,https://www.tucapital.es/depositos/depositos-d...,22/04/2024
3,El futuro de Bitcoin: predicciones para los pr...,https://www.tucapital.es/blogs/guias/el-futuro...,22/04/2024
4,Eco: Cetelem sólo permite transferencias máxim...,https://www.tucapital.es/blogs/eco/eco-cetelem...,19/04/2024


In [50]:
# Define a lambda function that uses Selenium and BeautifulSoup to get the text
def get_text_from_url(url):
    # Check for None URL
    if url is None or pd.isna(url):
        return None
    time.sleep(0.9)
    try:
        browser.get(url)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        content_wrapper = soup.find('div', id='content_wrapper')
        if content_wrapper:
            text = '\n'.join(p.get_text().strip() for p in content_wrapper.find_all('p')) # extract text from p tag
            return text
        else:
            return "content_wrapper not found"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# We'll apply the function to the 'url' column and create a new 'text' column
# Initialize tqdm progress bar
tqdm.pandas(desc="Scraping text")

# Apply the function using progress_apply from tqdm
df['text'] = df['url'].progress_apply(get_text_from_url)

# Close the WebDriver when done
#browser.quit()

# Display or save your DataFrame
df.head()


Scraping text: 100%|██████████| 1025/1025 [41:22<00:00,  2.42s/it]


,title,url,date,text
0,"Mañana, último día de la cuenta al 2,89% TAE y...",https://www.tucapital.es/depositos/manana-ulti...,23/04/2024,"\nMañana, día 24 de abril de 2024, se termina ..."
1,ING te da hasta 120€ si domicilias nómina y pa...,https://www.tucapital.es/cuentas/ing-te-da-has...,22/04/2024,"\nING ahora pide, además de domiciliar una nóm..."
2,Depósitos de Banco BIG: ligera subida en los n...,https://www.tucapital.es/depositos/depositos-d...,22/04/2024,\nBanco BIG remodela su oferta de depósitos co...
3,El futuro de Bitcoin: predicciones para los pr...,https://www.tucapital.es/blogs/guias/el-futuro...,22/04/2024,"Desde su lanzamiento, Bitcoin ha experimentado..."
4,Eco: Cetelem sólo permite transferencias máxim...,https://www.tucapital.es/blogs/eco/eco-cetelem...,19/04/2024,"Con motivo de la desaparición de Orange Bank, ..."


In [51]:
with_text_save_path = "/tucapital-news_with_text.csv"
df.to_csv(dfolder + with_text_save_path, index=False)